### Utilisation du réseau de neurone décrit : https://medium.com/@mohamedalihabib7/brain-tumor-detection-using-convolutional-neural-networks-30ccef6612b0

In [4]:
from PIL import Image
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset

device = torch.device("cuda:0")

### Lecture des données

In [5]:
list_img = []
labels = []

n_y = 0
n_n = 0
#print(os.getcwd())
directory = '../../new_data_norm2/'
for name in os.listdir(directory + 'yes'):
    img = Image.open(directory + 'yes/'+ name)
    img = img.resize((240,240))
    list_img.append(np.asarray(img).T)
    labels.append(1)
    n_y += 1
    
for name in os.listdir(directory + '/no'):
    img = Image.open(directory + 'no/'+ name)
    img = img.resize((240,240))
    list_img.append(np.asarray(img).T)
    labels.append(0)
    n_n += 1
    
#print(n_y, n_n)

In [6]:
list_img[4].shape

(3, 240, 240)

### Dataset Pytorch

In [7]:
class brain_Dataset(Dataset):
    def __init__ (self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data[1])
        
    def __getitem__(self, index):
        img = self.data[0][index]
        label = self.data[1][index]
        
        return img, label

In [8]:
dataset = brain_Dataset([list_img, labels])

In [9]:
batch_size = 25

data_size = len(list_img)
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
valloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         sampler=val_sampler)

### CNN

In [10]:
class brainCNN(nn.Module):
    def __init__(self):
        super(brainCNN, self).__init__()
        
        self.zp = nn.ConstantPad2d(2, 0)
        
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=(7,7)),
            nn.BatchNorm2d(32),
            nn.ReLU())
        
        self.mp1 = nn.MaxPool2d(4, stride=4)
        self.mp2 = nn.MaxPool2d(4, stride=4)
        self.ft = nn.Flatten()
        
        self.hidden1 = nn.Sequential(
            nn.Linear(6272, 2),
            nn.ReLU())
        self.sig = nn.Sigmoid()        
    
    def forward(self, x):
        x = self.zp(x)
        x = self.conv(x)
        x = self.mp1(x)
        x = self.mp2(x)
        x = self.ft(x)
        x = self.hidden1(x)
        x = self.sig(x)
        
        return x

### Entrainement

##### Obtention de bon résultat avec beaucoup d'epochs (~20) sur les données augmentés et crop

In [11]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            x_gpu = x.to(device, dtype=torch.float)
            y_gpu = y.to(device, dtype=torch.long)
            prediction = model(x_gpu)   
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)

            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    model.eval() # Evaluation mode
    
    correct_samples = 0
    total_samples = 0
    
    for i_step, (x, y) in enumerate(loader):
      x_gpu = x.to(device, dtype=torch.float)
      y_gpu = y.to(device, dtype=torch.long)
      predictions = model(x_gpu)
      _, indices = torch.max(predictions, 1)
      correct_samples += torch.sum(indices == y_gpu)
      total_samples += y.shape[0]
      
    accuracy = float(correct_samples)/total_samples       
    
    return accuracy

In [24]:
model = brainCNN()
model.type(torch.cuda.FloatTensor)
model.to(device)

loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(model.parameters(), lr =1e-4)

In [28]:
loss_history, train_history, val_history = train_model(model, trainloader, valloader, loss, optimizer, 150)

Average loss: 0.496509, Train accuracy: 0.980843, Val accuracy: 0.864450
Average loss: 0.497067, Train accuracy: 0.977650, Val accuracy: 0.859335
Average loss: 0.496537, Train accuracy: 0.980843, Val accuracy: 0.843990
Average loss: 0.498109, Train accuracy: 0.978927, Val accuracy: 0.861893
Average loss: 0.496840, Train accuracy: 0.978927, Val accuracy: 0.864450
Average loss: 0.496947, Train accuracy: 0.979566, Val accuracy: 0.859335
Average loss: 0.499146, Train accuracy: 0.980204, Val accuracy: 0.864450
Average loss: 0.496927, Train accuracy: 0.979566, Val accuracy: 0.867008
Average loss: 0.497590, Train accuracy: 0.981481, Val accuracy: 0.841432
Average loss: 0.498418, Train accuracy: 0.978289, Val accuracy: 0.854220
Average loss: 0.499922, Train accuracy: 0.980204, Val accuracy: 0.849105
Average loss: 0.495687, Train accuracy: 0.980204, Val accuracy: 0.856777
Average loss: 0.496399, Train accuracy: 0.979566, Val accuracy: 0.851662
Average loss: 0.495169, Train accuracy: 0.980843, V